# Estimating Gentrification using Street View Images and Embeddings

This script (initially produced by ChatGPT) does the following (_this was my query_):
 - Read a spatial boundary file (that I will hard code)
 - Obtain the road network (from OSM?) for that area
 - Generate sample points on the road network roughly X meters apart
 - At each sample point, download the most recent street images for that location (either a single 360 degree view of a few smaller images). Use whichever API service is the most appropriate for obtaining the images. Importantly please record the date that the image was taken.
 - For each image, calculate an embedding using an appropriate foundation model (one that has been pre-trained to distinguish street environments specifically). Please use Hugging Face libraries.
 - If necessary, calculate the mean embedding for each point (is this the best way to calculate a single embedding for a point represented by multiple images?)
 - Now, for each sampled point there will be a dataframe with information about the point and its embedding. Read another polygon spatial data file, that I will provide, which contains area-level estimates of gentrification.
 - Use point-in-polygon to get the gentrification for each point.
 - Use cross-validation to train a couple of ML models (probaly random forest, linear regression and a neural network) to estimate gentrification from the embedding vectors
 - Choose the best model and parameter configuration and test this model on some held-out data.

## Configuration and library loading

In [1]:
import os
os.environ["HF_HOME"] = "/nfs/a319/gy17m2a/scratch/hf_cache"
import random
import pickle
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import osmnx as ox
import torch
from PIL import Image
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from scipy.stats import entropy
import clip

# # Hugging Face Transformers for image embedding
from transformers import AutoImageProcessor, AutoModel, ViTModel, ViTImageProcessor, ViTConfig

# # ----------------- Configuration -----------------
# #np.random.seed(42)
# density_per_km = 0.3  # number of points to sample per km of road
# #density_per_km = 0.1  # VERY FEW WHILE TESTING
DOWNLOAD_IMAGES = False  # If false then don't download any images, just load those that have been cached

data_dir = os.path.join("../../data/embeddings/")
boundary_file = os.path.join(data_dir, "greater_manchester_lsoas.geojson")  # Path to boundary polygon file
lsoas_file = os.path.join("../../data/SpatialData/", "LSOAs_2021", "LSOA_2021_EW_BSC_V4.shp")
n_directions = 4         # number of street view images per point (e.g., 4 cardinal directions)

## Data Loading

In [2]:
# --- Load neighbourhood polygons and dissolve to one study-area boundary ---
boundary_neighs = gpd.read_file(boundary_file)
# Make sure we're in WGS84 (lat/lon) for OSM and APIs
boundary_neighs = boundary_neighs.to_crs(epsg=4326)
# Dissolve: merge all geometries into one polygon (MultiPolygon possible)
boundary_polygon = boundary_neighs.union_all()  # shapely (multi)polygon
boundary_gdf = gpd.GeoDataFrame(data={'name': ['study_area']},
    geometry=[boundary_polygon], crs=boundary_neighs.crs)
print("Merged neighbourhoods into single study-area boundary.")
print("Bounds:", boundary_polygon.bounds)
# boundary_gdf.plot(color='lightblue', edgecolor='black')

Merged neighbourhoods into single study-area boundary.
Bounds: (-2.73052481406758, 53.3281053809015, -1.90962093258169, 53.6857339236763)


## Read the LSOA boundary data

(later it will be joined to the Greater Manchester Gentrification Index and IMD)

In [3]:
lsoas =  gpd.read_file(lsoas_file)
manc_lads = ['Manchester', 'Rochdale', 'Bolton', 'Bury', 'Wigan', 'Oldham',  'Trafford', 'Salford', 'Tameside', 'Stockport']
manc_lads_pattern = '|'.join(manc_lads)
gm_lsoa=lsoas[lsoas['LSOA21NM'].str.contains(manc_lads_pattern)]
gm_lsoa = gm_lsoa.to_crs(epsg=4326)
# gm_lsoa.plot()

### Load list of sample points

In [4]:
# Cache file for the entire points data with embeddings (images are stored separately)
DEBUG = False
points_data_cache = data_dir + "sample_points_cache/points_data_cache_with_embeddings.pkl"

# -----------------------------------------------------------
# Load existing cache so we can *append* new sample points
# -----------------------------------------------------------
if os.path.isfile(points_data_cache):
    print("Loading cached point data …")
    with open(points_data_cache, "rb") as f:
        point_records = pickle.load(f)
    existing_coords = {(rec["latitude"], rec["longitude"]) for rec in point_records}
    next_id = max(rec["point_id"] for rec in point_records) + 1
else:
    point_records = []
    existing_coords = set()
    next_id = 0

print(f"Cache currently has {len(point_records)} points.")
added_this_run = 0

Loading cached point data …
Cache currently has 18897 points.


### Map of the full sample (cache + any others just downlaoded)

In [ ]:
# if point_records:    
#     fig, ax = plt.subplots(figsize=(8, 8))

#     # Plot roads first (thin gray lines)
#     # edges_gdf.plot(ax=ax, linewidth=0.1, color="gray")

#     # Plot the sample points
#     gpd.GeoDataFrame(point_records, 
#                      geometry=[Point(rec["longitude"], rec["latitude"]) for rec in point_records], 
#                      crs="EPSG:4326").plot(ax=ax, color="blue", markersize=0.2, label="Sample Points")

#     # Plot the study-area outline on top (thicker red line)
#     boundary_gdf.boundary.plot(ax=ax, linewidth=2, edgecolor="red")

#     ax.set_title("Road network & study-area boundary", pad=12)
#     ax.set_axis_off()          # hides lat/lon ticks for a cleaner look
#     ax.set_aspect("equal")     # keeps the map from looking stretched

#     plt.show()

## Compute the Embeddings

(Note: would like to use Places365 but not available in Hugging Face yet, so using ViT base model instead)

In [14]:
# ------------------------------
# 1. Load CLIP
# ------------------------------
device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# ------------------------------
# 2. Define categories
# ------------------------------
categories = [
    "a photo taken indoors, inside a building or a car",
    "a photo primarily of a terraced house",
    "a photo primarily of a detached or semi detached house",
    "a photo primarily of a road",
    "a photo of a shop",
    "a photo dominated by the outside of a car",
    "a photo of an industrial building",
    "a photo of wasteland, empty space",
    "a photo of nice green space, e.g. parks, gardens, nice trees and flowers"]

text_tokens = clip.tokenize(categories).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)  # normalize once


# ------------------------------
# 3. CLIP embedding + scoring
# ------------------------------
def embed_and_score_clip(img_path):
    """Return CLIP embedding (512d) + category similarity scores."""
    img = Image.open(img_path).convert("RGB")
    image_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        image_feat = model.encode_image(image_tensor)
        image_feat /= image_feat.norm(dim=-1, keepdim=True)  # normalize for cosine sim

        # similarities → shape (1, num_categories)
        sims = (image_feat @ text_features.T).softmax(dim=-1)

    return (
        image_feat.cpu().numpy()[0],      # 512-dim embedding
        sims.cpu().numpy()[0]             # category probs (len 9)
    )


# ------------------------------
# 4. Dummy fields for compatibility
# ------------------------------
def empty_attention_stats():
    return {"mean": None, "std": None, "entropy": None, "max_y": None, "max_x": None}


# ------------------------------
# 5. Embed + score all images
# ------------------------------
for rec in tqdm(point_records, desc="Embedding points", unit="point"):

    rec["embedding"] = []
    rec["category_scores"] = []      # <—— NEW
    rec["attn_stats"] = []           # empty placeholders
    rec["heatmap_paths"] = []        # always None

    for img_path in rec["image_files"]:
        img_path = (
            img_path.replace("airbnb-manchester/", "embeddings/")
                    .replace("../", "../../")
        )

        if not os.path.exists(img_path):
            continue

        try:
            embedding, scores = embed_and_score_clip(img_path)

            rec["embedding"].append(embedding)
            rec["category_scores"].append(scores)
            rec["attn_stats"].append(empty_attention_stats())
            rec["heatmap_paths"].append(None)

        except Exception as e:
            tqdm.write(f"⚠️ Error processing {img_path}: {e}")


# ------------------------------
# 6. Save cache
# ------------------------------
output_file = (
    data_dir + "sample_points_cache/points_data_cache_with_CLIP_embeddings_and_scores.pkl"
)

with open(output_file, "wb") as f:
    pickle.dump(point_records, f)

print(f"\n💾 Saved embeddings + category scores for {len(point_records)} points.")


Embedding points: 100%|███████████████████████████████████████████████████████████████████████████| 18897/18897 [52:46<00:00,  5.97point/s]



💾 Saved embeddings + category scores for 18897 points.


In [26]:
# Check all records have an embeding
invalid_records = [rec for rec in point_records if rec.get('embedding') is None]
assert len(invalid_records)==0, f"Found {len(invalid_records)} invalid points"